# 1 Let's try to play with data

In [8]:
import gzip
import tqdm.notebook as tqdm
from collections import defaultdict
import json

f = open("../data/goodreads_reviews_spoiler.json")
lines = []
for i in tqdm.tqdm(range(1378033)):
    lines.append(json.loads(f.readline()))

In [12]:
len(lines)

1378033

In [91]:
lines[0]

{'user_id': '8842281e1d1347389f2ab93d60773d4d',
 'timestamp': '2017-08-30',
 'review_sentences': [[0, 'This is a special book.'],
  [0,
   'It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind.'],
  [0,
   'This is what I love about good science fiction - it pushes your thinking about where things can go.'],
  [0,
   "It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I've read."],
  [0,
   'For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc.'],
  [0, 'It is a book about science, and aliens.'],
  [0,
   'The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell.'],
  [1,
   'Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think a

In [19]:
import numpy as np
n_spoiler_reviews = np.sum([line['has_spoiler'] for line in lines])
n_spoiler_reviews, n_spoiler_reviews/len(lines)

(89627, 0.06503980673902585)

In [21]:
import numpy as np
n_spoiler_sentences = np.sum([sentence[0] for line in lines for sentence in line['review_sentences']])
n_sentences = np.sum([len(line['review_sentences']) for line in lines])
n_spoiler_sentences, n_sentences, n_spoiler_sentences/n_sentences

(569724, 17672655, 0.03223760097167064)

In [22]:
sentences = [sentence[1] for line in lines for sentence in line['review_sentences']]
labels = [sentence[0] for sentence in sentences]

In [84]:
import string
from nltk.stem.porter import *

stemmer = PorterStemmer()
punctuation = set(string.punctuation)

def clean_review(sentence):
    return ''.join([c for c in sentence.lower() if c not in punctuation])

words = defaultdict(int)
    
for line in tqdm.tqdm(lines):
    for sentence in line['review_sentences']:
        sentence = clean_review(sentence[1])
        if (sentence != '') and (sentence is not None):
            for word in sentence.split():
                words[word] += 1

In [85]:
words['this']

3468057

Let's stem these words.

In [86]:
stem_words = defaultdict(int)
for word in tqdm.tqdm(list(words.keys())):
    stem_word = stemmer.stem(word)
    stem_words[stem_word] += words[word]

Let's look some high-frequency words.

In [89]:
counts = [(stem_words[w], w) for w in stem_words]
counts.sort()
counts.reverse()

counts[:20]

[(12689322, 'the'),
 (8411444, 'and'),
 (7832234, 'i'),
 (6832296, 'to'),
 (6524879, 'a'),
 (5653305, 'of'),
 (5082446, 'it'),
 (3898974, 'that'),
 (3687311, 'is'),
 (3661654, 'in'),
 (3514977, 'wa'),
 (3468597, 'thi'),
 (3132365, 'book'),
 (2491119, 'but'),
 (2272462, 'for'),
 (2082090, 'her'),
 (2049041, 'with'),
 (1789251, 'she'),
 (1624647, 'read'),
 (1615469, 'be')]

# 2 Let's do some basic statistics.

In [ ]:
pos_spoiler = []
pos_end_spoiler = []
pos_start_spoiler = []
length_spoiler = []
for review in lines:
    sentences = review['review_sentences']
    for sentence in sentences:
        